Can only start this when Marvin brings out a version that's compatible with the new OpenAI library.

In [1]:
from environs import Env
import openai
from marvin import ai_model
from pydantic import BaseModel, Field

env = Env()
env.read_env(".env")  # Read .env file
OPENAI_API_KEY = env("OPENAI_API_KEY")  # Get the API key
openai.api_key = OPENAI_API_KEY

In [21]:
@ai_model
class Plants_and_animals(BaseModel):
    """ A representation of all the plants and animals present in the sentence"""
    plants: list[str] = Field(description="The plants present in the sentence")
    animals: list[str] = Field(description= "The animals present in the sentence")
    sentence: str = Field(description= "the sentence itself")


Plants_and_animals("Of men neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twe handen vol Kamillen, of boter bloemen.")


[01/05/24 15:15:53] DEBUG    marvin.AIModel: Calling `ai_model` 'Plants_and_animals': with 'Of men    ]8;id=645303;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py\logging.py]8;;\:]8;id=998366;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py#65\65]8;;\
                             neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twe                  
                             handen vol Kamillen, of boter bloemen.'                                               

Plants_and_animals(plants=['kruid', 'Kamillen', 'boter bloemen'], animals=[], sentence='Of men neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twe handen vol Kamillen, of boter bloemen.')

In [24]:
import pandas as pd
output_gpt_and_bert = pd.read_csv("../data/llm_annotation/disagreement_analysis/round2/llm_and_ner_combination_round2.csv")
disagreements= output_gpt_and_bert.loc[output_gpt_and_bert["disagreement"]==True]


In [28]:
disagreements = disagreements.drop(columns =["Unnamed: 0.1", "Unnamed: 0", "flagged", "disagreement"])

In [54]:
import ast

def create_gpt4_prompt(row):
    def extract_entities(labels, entity_type, sentence):
        entities = []
        if isinstance(labels, list):
            for label in labels:
                if label[2] == entity_type:
                    # Extract the text snippet using the start and end indices
                    start, end = label[:2]
                    entity_text = sentence[start:end]
                    entities.append(entity_text)
        return entities

    sentence = row['sentence']

    try:
        gpt_labels = ast.literal_eval(row['label'])
        hf_labels = ast.literal_eval(row['huggingface_labels'])

        if not isinstance(gpt_labels, list):
            gpt_labels = []
        if not isinstance(hf_labels, list):
            hf_labels = []
    except (SyntaxError, ValueError):
        # Handle potential syntax errors or invalid data gracefully
        gpt_labels = []
        hf_labels = []

    gpt_plants = extract_entities(gpt_labels, 'plants', sentence)
    gpt_animals = extract_entities(gpt_labels, 'animals', sentence)
    hf_plants = extract_entities(hf_labels, 'plants', sentence)
    hf_animals = extract_entities(hf_labels, 'animals', sentence)

    prompt = (
        f"Sentence: '{sentence}'\n"
        f"Model1 (GPT) found the following plants: {gpt_plants}, and the following animals: {gpt_animals}.\n"
        f"Model2 (Hugging Face) found the following plants: {hf_plants}, and the following animals: {hf_animals}.\n")

    return prompt

# Apply the function to the DataFrame
disagreements['prompt'] = disagreements.apply(create_gpt4_prompt, axis=1)
prompts = disagreements['prompt'].tolist()


In [60]:
import tqdm
@ai_model
class Decide_on_annotations(BaseModel):
    """ 
    Two ai-models have been instructed to find all plants and animals in a sentence.
    In this case they disagreed, look at both options and give the right annotation.
    You are not obligtated to choose one of them, you can also choose a new one.
    """
    plants: list[str] = Field(description="The plants present in the sentence")
    animals: list[str] = Field(description= "The animals present in the sentence")
    sentence: str = Field(description= "the sentence itself")



a = prompts[0:10]

for e in tqdm.tqdm(a):
    b = Decide_on_annotations(e)
    print(b)





  0%|          | 0/10 [00:00<?, ?it/s]

[01/08/24 11:07:03] DEBUG    marvin.AIModel: Calling `ai_model` 'Decide_on_annotations': with         ]8;id=465532;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py\logging.py]8;;\:]8;id=119486;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py#65\65]8;;\
                             "Sentence: 'De jonge Ojevaars eindelyk genoegzaame kragten gekreegen                  
                             hebbende, vervoegen zich in de vlugt by de Ouden, en vertrekken in de                 
                             laatste dagen van Augustus.'\nModel1 (GPT) found the following plants:                
                             [], and the following animals: ['Ojevaars', 'Ouden'].\nModel2 (Hugging                
                             Face) found the following plants: [], and the following animals:                      
                             ['Ojevaars'].\n"                                                                      

 10%|█         | 1/10 [00:13<02:05, 13.95s/it]

plants=[] animals=['Ojevaars'] sentence='De jonge Ojevaars eindelyk genoegzaame kragten gekreegen hebbende, vervoegen zich in de vlugt by de Ouden, en vertrekken in de laatste dagen van Augustus.'


[01/08/24 11:07:17] DEBUG    marvin.AIModel: Calling `ai_model` 'Decide_on_annotations': with         ]8;id=346547;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py\logging.py]8;;\:]8;id=513401;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py#65\65]8;;\
                             "Sentence: 'De Oever-zwaluwen komen in Frankryk, en vertrekken bykans op              
                             denzelfden tyd als de Boeren-zwaluwen.'\nModel1 (GPT) found the                       
                             following plants: [], and the following animals: ['Oever-zwaluwen',                   
                             'Boeren-zwaluwen'].\nModel2 (Hugging Face) found the following plants:                
                             [], and the following animals: ['zwaluwen', 'zwaluwen'].\n"                           

...

 90%|█████████ | 9/10 [02:13<00:14, 14.36s/it]

plants=[] animals=['kudde'] sentence='Het werd eindelyk avond, de herders dreven hunne kudde naer huis en bliezen hun avondlied.'


[01/08/24 11:09:17] DEBUG    marvin.AIModel: Calling `ai_model` 'Decide_on_annotations': with         ]8;id=43462;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py\logging.py]8;;\:]8;id=679979;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py#65\65]8;;\
                             "Sentence: 'Men moet de Stonden doen komen, zo zy opgehouden zyn; Wat de              
                             Genees-kunst aangaat, men zal het zelfde, als in de verstopping der Milt              
                             aanmerken.'\nModel1 (GPT) found the following plants: [], and the                     
                             following animals: [].\nModel2 (Hugging Face) found the following                     
                             plants: [], and the following animals: ['Milt'].\n"                                   

100%|██████████| 10/10 [02:22<00:00, 14.26s/it]

plants=[] animals=[] sentence='Men moet de Stonden doen komen, zo zy opgehouden zyn; Wat de Genees-kunst aangaat, men zal het zelfde, als in de verstopping der Milt aanmerken.'


In [22]:
import re
from typing import List, Tuple, Union
from pydantic import BaseModel

class Annotations(BaseModel):
    sentence: str
    label: List[Tuple[Union[int, None], Union[int, None], Union[str, None]]] = []
    warning: bool = True
    additional_information: str = ""

    @classmethod
    def create_from_plants_and_animals(cls, data: Plants_and_animals):
        instance = cls(sentence=data.sentence)
        labels = []
        warning = False
        additional_info = ""

        for entity_type, entity_list in [("plant", data.plants), ("animal", data.animals)]:
            for plant_or_animal in entity_list:
                occurrences = [m.start() for m in re.finditer(plant_or_animal, data.sentence)]
                
                if len(occurrences) == 1:
                    start = occurrences[0]
                    end = start + len(plant_or_animal)
                    labels.append((start, end, entity_type))
                elif len(occurrences) > 1:
                    warning = True
                    additional_info = "multiple occurrences"
                else:
                    warning = True
                    additional_info = "entity could not be found"

        if warning:
            labels = [(None, None, None)]
            additional_info = "warning"

        instance.label = labels
        instance.warning = warning
        instance.additional_information = additional_info

        return instance


In [11]:
plants_and_animals_data = Plants_and_animals('Of men neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twee handen vol Kamillen, of boter bloemen.'
)

[01/05/24 15:02:25] DEBUG    marvin.AIModel: Calling `ai_model` 'Plants_and_animals': with 'Of men    ]8;id=401132;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py\logging.py]8;;\:]8;id=301112;file:///home/arjan_v_d/.local/lib/python3.11/site-packages/marvin/utilities/logging.py#65\65]8;;\
                             neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twee                 
                             handen vol Kamillen, of boter bloemen.'                                               

In [ ]:
plants_and_animals_data = Plants_and_animals(
    plants=['kruid', 'Kamillen', 'boter bloemen'],
    animals=[],
    sentence='Of men neemt drie handvollen van dat kruid, eer dat het gebloeid heeft, twe handen vol Kamillen, of boter bloemen.'
)

annotations = Annotations.create_from_plants_and_animals(plants_and_animals_data)
print(annotations.dict())

In [ ]:
from pydantic import BaseModel
from typing import List, Tuple

class EntitySpan(BaseModel):
    entity: str
    span: Tuple[int, int]

class SentenceAnalysis(BaseModel):
    plants: List[EntitySpan]
    animals: List[EntitySpan]
    warning: bool = False
    problem: str = ""

# Function to find all entity spans
def find_all_entity_spans(sentence: str, entity: str) -> List[Tuple[int, int]]:
    spans = []
    start = 0
    while start < len(sentence):
        start = sentence.find(entity, start)
        if start == -1:
            break
        end = start + len(entity)
        spans.append((start, end))
        start += len(entity)  # move past this entity
    return spans

sentence = "In de boom zit een aap, om preciezer te zijn: een baviaan"
Plants_and_animals(sentence)



# Validate and process response with Pydantic
try:
    analysis_result = SentenceAnalysis(plants=[], animals=[])

    for plant in api_response.get('plants', []):
        spans = find_all_entity_spans(sentence, plant)
        if len(spans) > 1:
            analysis_result.warning = True
            analysis_result.problem += f"Multiple occurrences of '{plant}'. "
        analysis_result.plants.extend([EntitySpan(entity=plant, span=span) for span in spans])

    for animal in api_response.get('animals', []):
        spans = find_all_entity_spans(sentence, animal)
        if len(spans) > 1:
            analysis_result.warning = True
            analysis_result.problem += f"Multiple occurrences of '{animal}'. "
        analysis_result.animals.extend([EntitySpan(entity=animal, span=span) for span in spans])

except KeyError:
    analysis_result.warning = True
    analysis_result.problem = "Unclear input."

